# Regularization

Let's improve our understanding of what impacted **Titanic** passenger's chance of survival
- We will use logistic classifiers which are easy to interpret
- Remember we already did it with statsmodels in lecture "Decision Science - Logistic Regression"
- We were using `p-values` & statistical assumptions to detect which features were irrelevant / don't generalize
- This time, we will use `regularization` to detect relevant/irrelevant features based on under/overfitting criteria
- **Our goal is to compare `L1` and `L2` penalties**

## 1. We load and preprocess the data for you

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/ML_titanic_dataset_encoded.csv")

# the dataset is already one-hot-encoded
data.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [3]:
# We build X and y

y = data["survived"]
X = data.drop(columns="survived")
X.head()

,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [4]:
# We MinMaxScale our features for you
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X)
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X.shape

(714, 12)

## 1.  Logistic Regression without regularization

❓ Rank the feature by decreasing order of importance according to a simple **non-regularized** Logistic Regression

- Careful, `LogisticRegression` is penalized by default
- Increase `max_iter` to a larger number until model converges

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

model = LogisticRegression(penalty='none')
model.fit(X_scaled, y)
pd.DataFrame(model.coef_.T, index=X.columns).sort_values(by=0, ascending=False)

,0
sex_female,2.671883
pclass,2.547187
class_First,2.360417
fare,1.358812
who_child,1.336356
parch,-0.893820
age,-2.196151
class_Third,-2.456891
sibsp,-2.477131
embark_town_Cherbourg,-11.221671


❓How do you interpret, in plain english language, the value for the coefficient `sex_female` ?

<details>
    <summary>Answer</summary>

> "All other things being equal (such as age, ticket class etc...),
being a women increases your log-odds of survival by 2.67 (your coef value)"
    
> "Controling for all other explaining factors available in this dataset,
being a women increases your odds-ratio of survival by exp(2.67) = 14"

</details>


❓ What is the feature that most impacts the chances of survival according to your model ? 

In [10]:
top_1_feature = ["embark_town_Queenstown"]

In [11]:
from nbresult import ChallengeResult
result = ChallengeResult('unregularized', top_1_feature = top_1_feature)
result.write()
print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/useradd/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/useradd/code/LucaVanTichelen/data-challenges/05-ML/05-Model-Tuning/02-Regularization
plugins: dash-2.0.0, anyio-3.3.2
collecting ... collected 1 item

tests/test_unregularized.py::TestUnregularized::test_top_1 PASSED        [100%]

============================== 1 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/unregularized.pickle

git commit -m 'Completed unregularized step'

git push origin master


## 2.  Logistic Regression with a L2 penalty

Let's use a **Logistic model** whose log-loss has been penalized with a **L2** term to figure out the **most important features** without overfitting.  
This is the "classification" equivalent to the "Ridge" regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its feature importance
- By "strongly regularized" we mean "more than sklearn's default applied regularization factor". 
- Default sklearn's values are very useful orders of magnitudes to keep in mind for "scaled features"

In [18]:
model = LogisticRegression(penalty='l2')
model.fit(X_scaled, y)
pd.DataFrame(model.coef_.T, index=X.columns).sort_values(by=0, ascending=False)

,0
sex_female,2.482402
who_child,1.126462
class_First,0.648990
fare,0.482459
embark_town_Cherbourg,0.260330
embark_town_Southampton,-0.116155
embark_town_Queenstown,-0.354136
parch,-0.539578
pclass,-0.715694
class_Third,-0.786230


❓ What are the top 2 features driving chances of survival according to your model ?

In [14]:
# Fill your top 2 features below
top_2_features = ['sex_female', 'who_child']

#### 🧪 Test your code below

In [15]:
from nbresult import ChallengeResult
result = ChallengeResult('ridge', top_2 = top_2_features)
result.write()
print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/useradd/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/useradd/code/LucaVanTichelen/data-challenges/05-ML/05-Model-Tuning/02-Regularization
plugins: dash-2.0.0, anyio-3.3.2
collecting ... collected 1 item

tests/test_ridge.py::TestRidge::test_top2 PASSED                         [100%]

============================== 1 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/ridge.pickle

git commit -m 'Completed ridge step'

git push origin master


## 2. Logistic Regression with a L1 penalty

This time, we'll use a logistic model whose log-loss has been penalized with a **L1** term to **filter-out the less important features**.  
This is the "classification" equivalent to the **Lasso** regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its feature importance

In [25]:
model = LogisticRegression(C=0.7, penalty='l1', solver='liblinear')
model.fit(X_scaled, y)
pd.DataFrame(model.coef_.T, index=X.columns).sort_values(by=0, ascending=False)

,0
sex_female,2.485419
who_child,1.021157
class_First,0.539110
embark_town_Cherbourg,0.333934
fare,0.000000
embark_town_Queenstown,0.000000
embark_town_Southampton,0.000000
parch,-0.007307
class_Third,-0.652604
pclass,-1.032200


❓ What are the features that have absolutely no impact on chances of survival, according to your L1 model?
- Do you notice how some of them were "highly important" according to the non-regularized model ? 
- From now on, we will always regularize our linear models!

In [26]:
zero_impact_features = ['fare', 'embark_town_Southampton', 'embark_town_Queenstown']

#### 🧪 Test your code below

In [27]:
from nbresult import ChallengeResult
result = ChallengeResult('lasso', zero_impact_features = zero_impact_features)
result.write()
print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/useradd/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/useradd/code/LucaVanTichelen/data-challenges/05-ML/05-Model-Tuning/02-Regularization
plugins: dash-2.0.0, anyio-3.3.2
collecting ... collected 1 item

tests/test_lasso.py::TestLasso::test_zero_impact PASSED                  [100%]

============================== 1 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/lasso.pickle

git commit -m 'Completed lasso step'

git push origin master


**🏁 Congratulation! Don't forget to commit and push your notebook**